# Algorytmy Tekstowe 2019/2020

# Laboratorium 6

# Autor - Łukasz Jezapkowicz

# 0. Przygotowanie danych

W tym punkcie wczytam pliki załączone do zadania. Są to odpowiednio pliki '1997_714.txt', 'romeo-i-julia-700.txt' oraz 'zad6'.  
  
W całym Notebooku dostępne będą one pod (odpowiednio) nazwami 'data1', 'data2' oraz 'data3'.

In [1]:
# otwieranie pliku
def open_file(file):
    file = open(file,mode='r', encoding="utf-8")
    data = file.read()
    file.close()
    return data

In [2]:
data1 = open_file('1997_714.txt')
data2 = open_file('romeo-i-julia-700.txt')
data3 = open_file('zad6')

In [3]:
print(data1[:500])





Dz.U. z 1998 r. Nr 144, poz. 930
                                       
                                       
                                       
                                       
                                    USTAWA
                          z dnia 20 listopada 1998 r.
                                       
         o zryczałtowanym podatku dochodowym od niektórych przychodów
                        osiąganych przez osoby fizyczne
                                       



In [4]:
print(data2[:500])

William Shakespeare

Romeo i Julia
tłum. Józef Paszkowski

ISBN 978-83-288-2903-9



OSOBY:
 * ESKALUS — książę panujący w Weronie
 * PARYS — młody Weroneńczyk szlachetnego rodu, krewny księcia
 * MONTEKI, KAPULET — naczelnicy dwóch domów nieprzyjaznych sobie
 * STARZEC — stryjeczny brat Kapuleta
 * ROMEO — syn Montekiego
 * MERKUCJO — krewny księcia
 * BENWOLIO — synowiec Montekiego
 * TYBALT — krewny Pani Kapulet
 * LAURENTY — ojciec franciszkanin
 * JAN — brat z tegoż zgromadzenia
 * BALTAZAR


In [5]:
print(data3)

1. Zaimplementować budowę słownika podstawowych składowych.
2. Zaimplementować wyszukiwanie wzorca korzystając z DBF. Wyszukiwanie ma działać dla każdej długości wzorca nie większej od długości całego tekstu. Czy konieczne jest budowanie nowego DBF dla pat&text?
3. Sprawdzić rzeczywisty czas budowy DBF dla załączonych plików. Porównać z czasem budowy drzewa sufiksów.*
4. Zbadać rzeczywisty rozmiar DBF, porównać z wielkością pliku.
5. Porównać czas wyszukiwania wzorca przy użyciu DBF z wyszukiwaniem za pomocą KMP dla różnych długości wzorca.**

* Osoby, które nie zaimplementowały drzewa sufiksów w poprzednich zadaniach mogą skorzystać np. z tej implementacji (w Pythonie): https://github.com/kvh/Python-Suffix-Tree
** j.w. dla algorytmu KMP: https://www.geeksforgeeks.org/kmp-algorithm-for-pattern-searching/

Rozwiązanie powinno zawierać kod oraz sprawozdanie w formacie PDF lub Jupyter Notebook.



# 1. Zaimplementować budowę słownika podstawowych składowych.

W poniższej implementacji stosuje symbolikę taką jak na wykładzie 5. Za słownik podstawowych składowych przyjmuję tablice Name oraz Pos dla wartości k będących potęgami liczby 2 nie większymi od długości tekstu w.  
  
W tym celu na początku zaimplementuję funkcję sort_rename, której zadaniem jest wyliczenie uporządkowanego etykietowania dla wejściowego ciągu S.

In [6]:
import itertools

In [7]:
# funkcja zwracająca pierwszy element krotki
def extract_key(el):
    return el[0]

# funkcja wyliczająca uporządkowane etykietowanie dla ciągu wejściowego S
# funkcja zwraca znalezione etykietowanie
def sort_rename(S):
    # dodawanie indeksów
    for i in range(1,len(S)+1):
        S[i-1] = (S[i-1],i)
    
    # sortowanie leksykograficzne
    S.sort()
    
    grouped = [[k,[x[1] for x in g]] for k, g in itertools.groupby(S, extract_key)]
    
    pos = []
    name = [None] * len(S)
    i = 1
    for element in grouped:
        pos.append(element[1])
        for index in element[1]:
            name[index-1] = i
        i += 1
            
    return name, pos

Test dla przykładowych danych (identycznych jak na wykładzie). Przyjąłem indeksowanie od 1. Warto zauważyć, że w tablicy $Pos$ przechowuje wszystkie indeksy dla danego fragmentu (co przyda mi się w późniejszym czasie).

In [8]:
X = [(1,2),(3,1),(2,2),(1,1),(2,3),(1,2)]
name, pos = sort_rename(X)
print('Name: ' + str(name))
print('Pos: ' + str(pos))

Name: [2, 5, 3, 1, 4, 2]
Pos: [[4], [1, 6], [3], [5], [2]]


Następnie zaimplementuję algorytm KMR, którego działanie opiera się na następującym fakcie:  
  
$Name_{2k}=sort\_rename(composite\_name_k)$  
  
$composite\_name_k[i]=(Name_k[i],Name_k[i+k])$  
  
Czyli możemy łatwo obliczać tablice Name i Pos dla kolejnych potęg 2.

In [9]:
import math

In [10]:
# funkcja realizująca algorytm KMR
def kmr(text):
    # kolejne tablice przechowywane w słowniku
    orig_length = len(text)
    names = dict()
    positions = dict()
    
    # dodawanie znaków # do tekstu
    factor = math.floor(math.log2(len(text)))
    text += '~' * (2 ** (factor+1) - 1)
    
    # tablice name i pos dla k = 1
    name, pos = sort_rename(list(text))
    names[1] = name[:orig_length]
    positions[1] = pos[:-1]
    
    # kolejne potęgi 2
    for i in range(1,factor+1):
        k = 2 ** i
        seq = []
        tmp = 2 ** (i-1)
        for j in range(orig_length):
            if (j + tmp < len(names[tmp])):
                seq.append((names[tmp][j],names[tmp][j+tmp]))
            else:
                seq.append((names[tmp][j],ord('~')))

        name, pos = sort_rename(seq)
        names[k] = name
        positions[k] = pos
                
    return names, positions

Przykład działania (z wykładu).

In [11]:
names, positions = kmr("abbabbaba")

print("names:")
for k,v in names.items():
    print(k,v)

print("\npositions:")
for k,v in positions.items():
    print(k,v)

names:
1 [1, 2, 2, 1, 2, 2, 1, 2, 1]
2 [1, 4, 3, 1, 4, 3, 1, 3, 2]
4 [2, 7, 5, 2, 7, 4, 1, 6, 3]
8 [3, 9, 6, 2, 8, 5, 1, 7, 4]

positions:
1 [[1, 4, 7, 9], [2, 3, 5, 6, 8]]
2 [[1, 4, 7], [9], [3, 6, 8], [2, 5]]
4 [[7], [1, 4], [9], [6], [3], [8], [2, 5]]
8 [[7], [4], [1], [9], [6], [3], [8], [5], [2]]


# 2. Zaimplementować wyszukiwanie wzorca korzystając z DBF. Wyszukiwanie ma działać dla każdej długości wzorca nie większej od długości całego tekstu. Czy konieczne jest budowanie nowego DBF dla pat&text?

Na początku zaimplementuję wersje algorytmu, który buduje DBF dla pat&text.  
  
Dla długości wzorca nie będącej potęgą dwójki możemy skorzystać z poniższej zalezności:  
  
$ Name_q[i]=Name_q[j] <=> (Name_t[i]=Name_t[j]\; and\; Name_t[i+q-t]=Name_t[j+q-t])$  
  
gdzie t jest największą potegą 2 mniejszą od długości wzorca.

In [12]:
from time import time
# funkcja implementująca wyszukiwanie wzorca korzystając z DBF
def pattern_dbf(text,pattern,print_results = True):
    if len(pattern) > len(text):
        print("Pattern is longer than text!")
        return
    time1 = time()
    names, positions = kmr(pattern + '&' + text)
    
    count = 0
    # długość wzorca to potęga 2
    if len(pattern) in names != None:
        name = names[len(pattern)]
        pat_lab = name[0]
        
        # omijam niepotrzebne elementy (pattern + &)
        for i in range(len(pattern)+1,len(name)):
            if name[i] == pat_lab:
                if print_results:
                    print("Pattern found starting at index " + str(i-len(pattern)-1))
                count += 1
    else:
        # największa składowa mniejsza od len(pattern) będąca potęgą 2
        smaller_factor = math.floor(math.log2(len(pattern)))
        k = 2 ** smaller_factor
        name = names[k]
        diff = len(pattern)-k
        
        # omijam niepotrzebne elementy i korzystam z powyższej zależności
        for i in range(len(pattern)+1,len(name)):
            if name[0] == name[i] and name[diff] == name[i + diff]:
                if print_results:
                    print("Pattern found starting at index " + str(i-len(pattern)-1))
                count += 1
        
    print("Total number of patterns found : " + str(count))
    if not print_results:
        print('It took ' + str(time()-time1) + ' seconds to find all occurences of given pattern in given text.')

Przykład działania poniżej:

In [13]:
# wyszukiwanie wzorca będącego pojedynczym symbolem (spodziewamy się 6 liter a)
pattern_dbf("ababababbaba","a")

Pattern found starting at index 0
Pattern found starting at index 2
Pattern found starting at index 4
Pattern found starting at index 6
Pattern found starting at index 9
Pattern found starting at index 11
Total number of patterns found : 6


In [14]:
# wyszukiwanie wzorca będącego potęgą 2 (spodziewamy się 5 wystąpień ab)
pattern_dbf("ababababbaba","ab")

Pattern found starting at index 0
Pattern found starting at index 2
Pattern found starting at index 4
Pattern found starting at index 6
Pattern found starting at index 9
Total number of patterns found : 5


In [15]:
# wyszukiwanie wzorca nie będącego potęga 2 (spodziewamy się 4 wystąpień aba)
pattern_dbf("ababababbaba","aba")

Pattern found starting at index 0
Pattern found starting at index 2
Pattern found starting at index 4
Pattern found starting at index 9
Total number of patterns found : 4


In [16]:
# wyszukiwanie wzorca będącego tekstem (spodziewamy się 1 wystąpienia)
pattern_dbf("ababababbaba","ababababbaba")

Pattern found starting at index 0
Total number of patterns found : 1


In [17]:
# wyszukiwanie wzorca dłuższego niż tekst (nie powinniśmy niczego szukać)
pattern_dbf("ababababbaba","ababababbabaa")

Pattern is longer than text!


### Czy można bez pattern&text?
  
Warto zauważyć, że w powyższej implementacji w ogóle nie korzystamy z tablicy $Pos$. Można więc sądzić, że nie wykorzystujemy całości informacji dostarczonych przez $KMR$.  
  
Poszczególne tablice $Pos[i]$ zawierają numery indeksów rozpoczynających kolejne fragmenty (tablica jest uporządkowana leksykograficznie względem fragmentów - kluczowa informacja!).  
  
Moglibyśmy zatem dla wzorca o długości $k$ wziąć pod uwagę tablicę $Pos[i]$, gdzie $i\;=\;max\;\{j\;:2^j\;<= k\}$ (czyli najdłuższy możliwy prefiks naszego wzorca). W owej tablicy moglibyśmy użyć wyszukiwania binarnego w celu znalezienia czy istnieją wystąpienia naszego wzorca (tablica jest uporządkowana). Zależnie od tego czy k jest potęgą dwójki algorytm dzieliłby się na dwie metodyki:
  
- jeśli jest, to porównujemy cały wzorzec z każdym fragmentem zaczynającym się w danym fragmencie. Jeśli znajdziemy listę indeksów, w której pierwszy element pasuje to cała lista jest szukanym rozwiązaniem.
  
- jeśli nie jest, to porównujemy tylko fragmenty o długości $i$ i jeżeli znajdziemy listę, w której porównanie zakończyło się pozytywnie, to sprawdzamy każdy jej element tak jak w poprzedniej metodzie (dla przykładu, szukając wzorca $abb$ i biorąc listę $[ab,ab,ab]$ i odpowiadające jej fragmenty $[aba,abb,abb]$ moglibyśmy odrzucić pasujące rozwiązania gdybyśmy postępowali jak powyżej).
  
Złożoność takiego rozwiązania (nie przyjmując żadnych pesymistycznych sytuacji, wyjdzie w praniu) to $O(k*logn)$. Jak widać jest to dosyć obiecująca złożoność, zobaczymy jak wyjdą wyniki.
  
Poniżej znajduje się implementacja wyżej wymienionego pomysłu.

In [18]:
# funkcja porównująca dwa fragmenty tekstu (leksykograficznie)
def compare_chunks(chunk1, chunk2, length):
    if len(chunk1) < length:
        chunk1 += '~' * (length - len(chunk1))
        
    t1 = chunk1[:length]
    t2 = chunk2[:length]
    if t1 > t2:
        return 1
    elif t1 < t2:
        return -1
    else:
        return 0

In [19]:
# funkcja zwracająca szukaną listę lub None, gdy takiej nie ma
def find_list(positions, text, pattern, length):
    i = 2 ** (math.floor(math.log2(length))) # szukany indeks
    l, r, mid = 0, len(positions[i]) - 1, length / 2
    pos = positions[i]
    while l <= r: 
        mid = (l + r) // 2
        cur_pos = pos[mid][0]-1
        comp = compare_chunks(text[cur_pos:(cur_pos+length)],pattern,length)
        if comp == 1:
            r = mid - 1
        elif comp == -1:
            l = mid + 1
        else:
            return pos[mid]
        
    return None

In [20]:
# funkcja znajdująca wzorzec przy pomocy gotowego DBF
def better_pattern_dbf(dbf,text,pattern,print_results=True):
    if len(pattern) > len(text):
        print("Pattern is longer than text!")
        return
    time1 = time()
    # najpierw poszukujemy naszej listy
    indexes_list = find_list(dbf[1],text,pattern,len(pattern))
    
    if indexes_list is not None:
        if len(pattern) == 2 ** (math.floor(math.log2(len(pattern)))): # jest potęgą 2
            for index in indexes_list:
                if print_results:
                    print("Pattern found starting at index " + str(index-1))
            print("Total number of patterns found : " + str(len(indexes_list)))
        else:
            count = 0
            for index in indexes_list:
                if pattern == text[index-1:index-1+len(pattern)]:
                    if print_results:
                        print("Pattern found starting at index " + str(index-1))
                    count += 1
            print("Total number of patterns found : " + str(count))
    if not print_results:
        print('It took ' + str(time()-time1) + ' seconds to find all occurences of given pattern in given text.')

Przetestujmy teraz działanie naszego algorytmu (narazie bez mierzenia czasów).

In [21]:
text = "ababababbaba"
dbf = kmr(text)

In [22]:
# wyszukiwanie wzorca będącego pojedynczym symbolem (spodziewamy się 6 liter a)
better_pattern_dbf(dbf,text,"a")

Pattern found starting at index 0
Pattern found starting at index 2
Pattern found starting at index 4
Pattern found starting at index 6
Pattern found starting at index 9
Pattern found starting at index 11
Total number of patterns found : 6


In [23]:
# wyszukiwanie wzorca będącego potęgą 2 (spodziewamy się 5 wystąpień ab)
better_pattern_dbf(dbf,text,"ab")

Pattern found starting at index 0
Pattern found starting at index 2
Pattern found starting at index 4
Pattern found starting at index 6
Pattern found starting at index 9
Total number of patterns found : 5


In [24]:
# wyszukiwanie wzorca nie będącego potęga 2 (spodziewamy się 4 wystąpień aba)
better_pattern_dbf(dbf,text,"aba")

Pattern found starting at index 0
Pattern found starting at index 2
Pattern found starting at index 4
Pattern found starting at index 9
Total number of patterns found : 4


In [25]:
# wyszukiwanie wzorca będącego tekstem (spodziewamy się 1 wystąpienia)
better_pattern_dbf(dbf,text,text)

Pattern found starting at index 0
Total number of patterns found : 1


In [26]:
# wyszukiwanie wzorca dłuższego niż tekst (nie powinniśmy niczego szukać)
better_pattern_dbf(dbf,text,"ababababbabaa")

Pattern is longer than text!


Jak widać wyniki są identyczne jak dla wersji z budowaniem nowego $DBF$. Algorytm działa zatem poprawnie. To jak algorytm spisuje się czasowo przetestujemy później.

# 3. Sprawdzić rzeczywisty czas budowy DBF dla załączonych plików (3 pliki). Porównać z czasem budowy drzewa sufiksów.

Poniżej wersja KMR z dodanym mierzeniem czasu (jedyna zmiana w kodzie).

In [27]:
# funkcja realizująca algorytm KMR oraz zliczająca czas budowy
def kmr_time(text):
    time1 = time()
    # kolejne tablice przechowywane w słowniku
    orig_length = len(text)
    names = dict()
    positions = dict()
    
    # dodawanie znaków # do tekstu
    factor = math.floor(math.log2(len(text)))
    text += '~' * (2 ** (factor+1) - 1)
    
    # tablice name i pos dla k = 1
    name, pos = sort_rename(list(text))
    names[1] = name[:orig_length]
    positions[1] = pos[:-1]
    
    # kolejne potęgi 2
    for i in range(1,factor+1):
        k = 2 ** i
        seq = []
        tmp = 2 ** (i-1)
        for j in range(orig_length):
            if (j + tmp < len(names[tmp])):
                seq.append((names[tmp][j],names[tmp][j+tmp]))
            else:
                seq.append((names[tmp][j],ord('~')))

        name, pos = sort_rename(seq)
        names[k] = name
        positions[k] = pos
    
    print('It took ' + str(time()-time1) + ' seconds to build DBF from given text.')
    return names, positions, time()-time1

Poniżej czas budowania DBF dla kolejnych plików.

In [28]:
times_dbf = []

In [29]:
times_dbf.append(kmr_time(data1)[2])

It took 25.63781499862671 seconds to build DBF from given text.


In [30]:
times_dbf.append(kmr_time(data2)[2])

It took 0.599107027053833 seconds to build DBF from given text.


In [31]:
times_dbf.append(kmr_time(data3)[2])

It took 0.024932861328125 seconds to build DBF from given text.


Teraz sprawdzę czas budowania drzewa sufiksów (użyłem danej implementacji, moja była trochę wadliwa).

In [32]:
class Node(object):
    """A node in the suffix tree. 
    
    suffix_node
        the index of a node with a matching suffix, representing a suffix link.
        -1 indicates this node has no suffix link.
    """
    def __init__(self):
        self.suffix_node = -1   

    def __repr__(self):
        return "Node(suffix link: %d)"%self.suffix_node

class Edge(object):
    """An edge in the suffix tree.
    
    first_char_index
        index of start of string part represented by this edge
        
    last_char_index
        index of end of string part represented by this edge
        
    source_node_index
        index of source node of edge
    
    dest_node_index
        index of destination node of edge
    """
    def __init__(self, first_char_index, last_char_index, source_node_index, dest_node_index):
        self.first_char_index = first_char_index
        self.last_char_index = last_char_index
        self.source_node_index = source_node_index
        self.dest_node_index = dest_node_index
        
    @property
    def length(self):
        return self.last_char_index - self.first_char_index

    def __repr__(self):
        return 'Edge(%d, %d, %d, %d)'% (self.source_node_index, self.dest_node_index 
                                        ,self.first_char_index, self.last_char_index )


class Suffix(object):
    """Represents a suffix from first_char_index to last_char_index.
    
    source_node_index
        index of node where this suffix starts
    
    first_char_index
        index of start of suffix in string
        
    last_char_index
        index of end of suffix in string
    """
    def __init__(self, source_node_index, first_char_index, last_char_index):
        self.source_node_index = source_node_index
        self.first_char_index = first_char_index
        self.last_char_index = last_char_index
        
    @property
    def length(self):
        return self.last_char_index - self.first_char_index
                
    def explicit(self):
        """A suffix is explicit if it ends on a node. first_char_index
        is set greater than last_char_index to indicate this.
        """
        return self.first_char_index > self.last_char_index
    
    def implicit(self):
        return self.last_char_index >= self.first_char_index

        
class SuffixTree(object):
    """A suffix tree for string matching. Uses Ukkonen's algorithm
    for construction.
    """
    def __init__(self, string, case_insensitive=False):
        """
        string
            the string for which to construct a suffix tree
        """
        self.string = string
        self.case_insensitive = case_insensitive
        self.N = len(string) - 1
        self.nodes = [Node()]
        self.edges = {}
        self.active = Suffix(0, 0, -1)
        if self.case_insensitive:
            self.string = self.string.lower()
        for i in range(len(string)):
            self._add_prefix(i)
    
    def __repr__(self):
        """ 
        Lists edges in the suffix tree
        """
        curr_index = self.N
        s = "\tStart \tEnd \tSuf \tFirst \tLast \tString\n"
        values = list(self.edges.values())
        values.sort(key=lambda x: x.source_node_index)
        for edge in values:
            if edge.source_node_index == -1:
                continue
            s += "\t%s \t%s \t%s \t%s \t%s \t"%(edge.source_node_index
                    ,edge.dest_node_index 
                    ,self.nodes[edge.dest_node_index].suffix_node 
                    ,edge.first_char_index
                    ,edge.last_char_index)
                    
            
            top = min(curr_index, edge.last_char_index)
            s += self.string[edge.first_char_index:top+1] + "\n"
        return s
            
    def _add_prefix(self, last_char_index):
        """The core construction method.
        """
        last_parent_node = -1
        while True:
            parent_node = self.active.source_node_index
            if self.active.explicit():
                if (self.active.source_node_index, self.string[last_char_index]) in self.edges:
                    # prefix is already in tree
                    break
            else:
                e = self.edges[self.active.source_node_index, self.string[self.active.first_char_index]]
                if self.string[e.first_char_index + self.active.length + 1] == self.string[last_char_index]:
                    # prefix is already in tree
                    break
                parent_node = self._split_edge(e, self.active)
        

            self.nodes.append(Node())
            e = Edge(last_char_index, self.N, parent_node, len(self.nodes) - 1)
            self._insert_edge(e)
            
            if last_parent_node > 0:
                self.nodes[last_parent_node].suffix_node = parent_node
            last_parent_node = parent_node
            
            if self.active.source_node_index == 0:
                self.active.first_char_index += 1
            else:
                self.active.source_node_index = self.nodes[self.active.source_node_index].suffix_node
            self._canonize_suffix(self.active)
        if last_parent_node > 0:
            self.nodes[last_parent_node].suffix_node = parent_node
        self.active.last_char_index += 1
        self._canonize_suffix(self.active)
        
    def _insert_edge(self, edge):
        self.edges[(edge.source_node_index, self.string[edge.first_char_index])] = edge
        
    def _remove_edge(self, edge):
        self.edges.pop((edge.source_node_index, self.string[edge.first_char_index]))
        
    def _split_edge(self, edge, suffix):
        self.nodes.append(Node())
        e = Edge(edge.first_char_index, edge.first_char_index + suffix.length, suffix.source_node_index, len(self.nodes) - 1)
        self._remove_edge(edge)
        self._insert_edge(e)
        self.nodes[e.dest_node_index].suffix_node = suffix.source_node_index  ### need to add node for each edge
        edge.first_char_index += suffix.length + 1
        edge.source_node_index = e.dest_node_index
        self._insert_edge(edge)
        return e.dest_node_index

    def _canonize_suffix(self, suffix):
        """This canonizes the suffix, walking along its suffix string until it 
        is explicit or there are no more matched nodes.
        """
        if not suffix.explicit():
            e = self.edges[suffix.source_node_index, self.string[suffix.first_char_index]]
            if e.length <= suffix.length:
                suffix.first_char_index += e.length + 1
                suffix.source_node_index = e.dest_node_index
                self._canonize_suffix(suffix)
 

    # Public methods
    def find_substring(self, substring):
        """Returns the index of substring in string or -1 if it
        is not found.
        """
        if not substring:
            return -1
        if self.case_insensitive:
            substring = substring.lower()
        curr_node = 0
        i = 0
        while i < len(substring):
            edge = self.edges.get((curr_node, substring[i]))
            if not edge:
                return -1
            ln = min(edge.length + 1, len(substring) - i)
            if substring[i:i + ln] != self.string[edge.first_char_index:edge.first_char_index + ln]:
                return -1
            i += edge.length + 1
            curr_node = edge.dest_node_index
        return edge.first_char_index - len(substring) + ln
        
    def has_substring(self, substring):
        return self.find_substring(substring) != -1

Poniżej czas budowania drzewa sufiksów dla kolejnych plików.

In [33]:
# funkcja mierząca czas budowy drzewa sufiksów
def suffix_time(data):
    time1 = time()
    st = SuffixTree(data)
    print('It took ' + str(time()-time1) + ' seconds to build Suffix Tree from given text.')
    return time()-time1

In [34]:
times_suffix = []

In [35]:
times_suffix.append(suffix_time(data1))

It took 2.719623327255249 seconds to build Suffix Tree from given text.


In [36]:
times_suffix.append(suffix_time(data2))

It took 0.10970735549926758 seconds to build Suffix Tree from given text.


In [37]:
times_suffix.append(suffix_time(data3))

It took 0.007976293563842773 seconds to build Suffix Tree from given text.


Poniżej porównanie czasów (UWAGA! Wymagana biblioteka tabulate (conda install -c conda-forge tabulate)):

In [39]:
from tabulate import tabulate

results = [(times_dbf[0],times_suffix[0]),(times_dbf[1],times_suffix[1]),(times_dbf[2],times_suffix[2])]
print(tabulate(results, headers=["DBF", "Suffix Tree"]))

       DBF    Suffix Tree
----------  -------------
25.6378        2.71962
 0.600104      0.109707
 0.0259304     0.00797629


#### Wniosek:
Wyniki nie powinny nas dziwić. Budowa drzewa sufiksów to złożoność $O(n)$ zaś budowa DBF to $O(n*logn)$ (dla Counting Sorta). Wraz ze wzrostem wielkości tekstu różnica między czasem budowy drzewa sufiksów a DBF powiększa się.  
  
Wykonane doświadczenie świadczy o nieliniowości algorytmu budowy DBF.

# 4. Zbadać rzeczywisty rozmiar DBF, porównać z wielkością pliku.

W podpunkcie tym skorzystałem z gotowej implementacji funkcji mierzącej rozmiar dowolnego obiektu w bajtach. Implementacja widoczna poniżej:

In [40]:
import sys
import inspect

def get_size(obj, seen=None):
    """Recursively finds size of objects in bytes"""
    size = sys.getsizeof(obj)
    if seen is None:
        seen = set()
    obj_id = id(obj)
    if obj_id in seen:
        return 0
    # Important mark as seen *before* entering recursion to gracefully handle
    # self-referential objects
    seen.add(obj_id)
    if hasattr(obj, '__dict__'):
        for cls in obj.__class__.__mro__:
            if '__dict__' in cls.__dict__:
                d = cls.__dict__['__dict__']
                if inspect.isgetsetdescriptor(d) or inspect.ismemberdescriptor(d):
                    size += get_size(obj.__dict__, seen)
                break
    if isinstance(obj, dict):
        size += sum((get_size(v, seen) for v in obj.values()))
        size += sum((get_size(k, seen) for k in obj.keys()))
    elif hasattr(obj, '__iter__') and not isinstance(obj, (str, bytes, bytearray)):
        size += sum((get_size(i, seen) for i in obj))
        
    if hasattr(obj, '__slots__'): # can have __slots__ with __dict__
        size += sum(get_size(getattr(obj, s), seen) for s in obj.__slots__ if hasattr(obj, s))
        
    return size

In [47]:
test1 = kmr(data1)
test2 = kmr(data2)
test3 = kmr(data3)

Rozmiary obiektów (DBF):

In [48]:
print(str(get_size(test1) / 1024) + ' KB')
print(str(get_size(test2) / 1024) + ' KB')
print(str(get_size(test3) / 1024) + ' KB')

609600.18359375 KB
25655.5078125 KB
1192.09375 KB


Rozmiary plików:

In [49]:
print(str(len(data1) / 1024) + ' KB')
print(str(len(data2) / 1024) + ' KB')
print(str(len(data3) / 1024) + ' KB')

240.6953125 KB
12.353515625 KB
0.8837890625 KB


Ile razy większy rozmiar ma obiekt od pliku?

In [50]:
print((get_size(test1)) / len(data1))
print((get_size(test2)) / len(data2))
print((get_size(test3)) / len(data3))

2532.663296439352
2076.7778656126484
1348.8441988950276


Ile razy dłuższy jest tekst x od tekstu 3 (najkrótszego)?

In [51]:
print(len(data1) / len(data3))
print(len(data2) / len(data3))
print(len(data3) / len(data3))

272.3447513812155
13.977900552486188
1.0


Ile razy więcej zajmuje obiekt DBF dla tekstu x od obiektu DBF dla tekstu 3 (najkrótszego)?

In [52]:
print((get_size(test1)) / get_size(test3))
print((get_size(test2)) / get_size(test3))
print((get_size(test3)) / get_size(test3))

511.3693311400634
21.521384381471677
1.0


#### Wniosek

Skąd wzięły się takie wyniki? Oczywistym faktem jest, że DBF będzie zajmowało więcej miejsca niż sam tekst (w końcu dla każdego $k = 2^i$ tablica Name ma długość równą długości tekstu + przechowuje inty a nie chary).  
  
Ilość tablic Name i Pos rośnie logarytmicznie. Długość każdej tablicy Name jest dla każdego tekstu stała i jest równa długości tekstu. Tablica Pos jednak zwiększa swoją długość (ponieważ jest więcej k-elementowych podciągów). Można podejrzewać więc, że rozmiar obiektu nie rośnie logarytmicznie do długości tekstu (albo rośnie liniowo albo logarytmicznie z dużą stałą). Przykładem tego jest fakt, że iloraz długości tekstu 1 oraz 3 jest mniejszy niż iloraz wielkości obiektu DBF dla tekstu 1 oraz 3 (długośc tekstu liniowo a iloczyn jednak jest mniejszy -> albo jest bardzo duża stała albo wielkość obiektu rośnie co najmniej liniowo).  
  
Przyjmujemy, że złożoność pamięciowa jest identyczna z czasową i wynosi $O(nlogn)$.  
  
Rozmiar $DBF$ jest zatem dużo większy od rozmiaru pliku.

# 5. Porównać czas wyszukiwania wzorca przy użyciu DBF z wyszukiwaniem za pomocą KMP dla różnych długości wzorca (np. jedna litera, jedno słowo, jedno zdanie).

W tym zadaniu użyję własnej implementacji KMP (z zadania numer 1). Implementacja widoczna poniżej:

In [53]:
# algorytm KMP
def KMPSearch(txt,pattern):
    time1 = time()
    S = set()
    K = []  
    t = -1
    K.append(t)
    for k in range(1, len(pattern) + 1):
        while(t >= 0 and pattern[t] != pattern[k - 1]):
            t = K[t]
        t = t + 1  
        K.append(t)
    time1 = time()
    m = 0  
    for i in range(0, len(txt)):
        while (m >= 0 and pattern[m] != txt[i]):
            m = K[m]
        m = m + 1  
        if m == len(pattern):
            S.add(i-m+1)
            m = K[m]
    print('It took ' + str(time()-time1) + ' seconds to find all occurences of given pattern in given text.')
    print("Total found occurences: " + str(len(S)))

In [54]:
# przygotowanie DBF dla wersji 2 algorytmu
dbf1 = kmr_time(data1)
dbf2 = kmr_time(data2)
dbf3 = kmr_time(data3)

It took 30.19686770439148 seconds to build DBF from given text.
It took 0.8457825183868408 seconds to build DBF from given text.
It took 0.03187894821166992 seconds to build DBF from given text.


In [252]:
text = data1
pattern = 'b'
KMPSearch(text,pattern)
print('====================')
pattern_dbf(text,pattern,False)
print('====================')
better_pattern_dbf(dbf1,text,pattern,False)

It took 0.08780980110168457 seconds to find all occurences of given pattern in given text.
Total found occurences: 1703
Total number of patterns found : 1703
It took 25.14922046661377 seconds to find all occurences of given pattern in given text.
Total number of patterns found : 1703
It took 0.0 seconds to find all occurences of given pattern in given text.


In [55]:
text = data1
pattern = 'Art.'
KMPSearch(text,pattern)
print('====================')
pattern_dbf(text,pattern,False)
print('====================')
better_pattern_dbf(dbf1,text,pattern,False)

It took 0.06482672691345215 seconds to find all occurences of given pattern in given text.
Total found occurences: 58
Total number of patterns found : 58
It took 29.972514867782593 seconds to find all occurences of given pattern in given text.
Total number of patterns found : 58
It took 0.0 seconds to find all occurences of given pattern in given text.


In [56]:
text = data1
pattern = 'nie wlicza się:'
KMPSearch(text,pattern)
print('====================')
pattern_dbf(text,pattern,False)
print('====================')
better_pattern_dbf(dbf1,text,pattern,False)

It took 0.07679367065429688 seconds to find all occurences of given pattern in given text.
Total found occurences: 1
Total number of patterns found : 1
It took 25.5556321144104 seconds to find all occurences of given pattern in given text.
Total number of patterns found : 1
It took 0.0 seconds to find all occurences of given pattern in given text.


In [57]:
text = data1
pattern = data1
KMPSearch(text,pattern)
print('====================')
pattern_dbf(text,pattern,False)
print('====================')
better_pattern_dbf(dbf1,text,pattern,False)

It took 0.0638132095336914 seconds to find all occurences of given pattern in given text.
Total found occurences: 1
Total number of patterns found : 1
It took 51.27433180809021 seconds to find all occurences of given pattern in given text.
Total number of patterns found : 1
It took 0.0040247440338134766 seconds to find all occurences of given pattern in given text.


Algorytm KMP posiada złożoność $O(n+k)$, gdzie $n$ i $k$ to odpowiednio długość tekstu oraz wzorca. Nasz pierwszy algorytm wyszukiwania wzorca ma złożoność $O(nlogn)$ gdzie $n$ to długość tekstu. Widać więc, że dla jakiegokolwiek dłuższego tekstu (data1) różnica czasu obliczeń może być kolosalna ($\frac{30.0}{0.07}\approx{428}$ razy dłużej!). Gdy jednak zastosujemy drugi algorytm używający przygotowanego wcześniej $DBF$ to czas wyszukiwania jest niemal niezauważalny! Sprawdźmy jak wygląda to dla krótszych tekstów (data2 oraz data3).

In [58]:
text = data2
pattern = 'ROMEO'
KMPSearch(text,pattern)
print('====================')
pattern_dbf(text,pattern,False)
print('====================')
better_pattern_dbf(dbf2,text,pattern,False)

It took 0.002991199493408203 seconds to find all occurences of given pattern in given text.
Total found occurences: 16
Total number of patterns found : 16
It took 0.7081522941589355 seconds to find all occurences of given pattern in given text.
Total number of patterns found : 16
It took 0.0 seconds to find all occurences of given pattern in given text.


In [59]:
text = data2
pattern = 'KAPULET'
KMPSearch(text,pattern)
print('====================')
pattern_dbf(text,pattern,False)
print('====================')
better_pattern_dbf(dbf2,text,pattern,False)

It took 0.0030336380004882812 seconds to find all occurences of given pattern in given text.
Total found occurences: 5
Total number of patterns found : 5
It took 0.7437288761138916 seconds to find all occurences of given pattern in given text.
Total number of patterns found : 5
It took 0.0 seconds to find all occurences of given pattern in given text.


In [60]:
text = data2
pattern = 'NIE ZNAJDZIECIE MNIE TUTAJ NIGDZIE'
KMPSearch(text,pattern)
print('====================')
pattern_dbf(text,pattern,False)
print('====================')
better_pattern_dbf(dbf2,text,pattern,False)

It took 0.0029649734497070312 seconds to find all occurences of given pattern in given text.
Total found occurences: 0
Total number of patterns found : 0
It took 0.7161164283752441 seconds to find all occurences of given pattern in given text.
It took 0.0 seconds to find all occurences of given pattern in given text.


Widać, że dla krótszego tekstu iloraz jest mniejszy ($\frac{0.70}{0.003}\approx{233}$). Czas dla drugiego algorytmu jest nadal niezauważalny.

In [61]:
text = data3
pattern = 'Zaimplementować'
KMPSearch(text,pattern)
print('====================')
pattern_dbf(text,pattern,False)
print('====================')
better_pattern_dbf(dbf3,text,pattern,False)

It took 0.0 seconds to find all occurences of given pattern in given text.
Total found occurences: 2
Total number of patterns found : 2
It took 0.027933835983276367 seconds to find all occurences of given pattern in given text.
Total number of patterns found : 2
It took 0.0 seconds to find all occurences of given pattern in given text.


In [62]:
text = data3
pattern = 'DBF'
KMPSearch(text,pattern)
print('====================')
pattern_dbf(text,pattern,False)
print('====================')
better_pattern_dbf(dbf3,text,pattern,False)

It took 0.0009982585906982422 seconds to find all occurences of given pattern in given text.
Total found occurences: 5
Total number of patterns found : 5
It took 0.024907350540161133 seconds to find all occurences of given pattern in given text.
Total number of patterns found : 5
It took 0.0 seconds to find all occurences of given pattern in given text.


In [63]:
text = data3
pattern = 'Algorytmy tekstowe już się powoli kończą'
KMPSearch(text,pattern)
print('====================')
pattern_dbf(text,pattern,False)
print('====================')
better_pattern_dbf(dbf3,text,pattern,False)

It took 0.0 seconds to find all occurences of given pattern in given text.
Total found occurences: 0
Total number of patterns found : 0
It took 0.03191494941711426 seconds to find all occurences of given pattern in given text.
It took 0.0 seconds to find all occurences of given pattern in given text.


Dla najmniejszego tekstu iloraz jest jeszcze mniejszy ($\frac{0.028}{0.0005}\approx{56}$). Widać więc, że czasowo algorytmy zrównają się być może dla tekstów wielkości jednego zdania lub mniejszych (nie ma większego sensu używać takiego algorytmu).  Nadal szybkościowo wygrywa nasz drugi algorytm.

#### Wnioski  
  
Słownik podstawowych składowych DBF ma wiele zastosowań. Jednym z nich jest wyszukiwanie wzorca $pattern$ w tekście $text$. Z wykonanego laboratorium płyną następujące wnioski:
  
- algorytm wyszukiwania wzorca działający na zasadzie tworzenia $DBF$ dla $pat\&text$ nie ma większego sensu, ponieważ $KMP$ jest zawsze szybszy oraz ma znacząco mniejsze wykorzystanie pamięciowe $O(pattern\_length)$.  
- algorytm wyszukiwania wzorca działający na zasadzie sprytnego wykorzystania tablic $Pos$ okazał się szybszy niż $KMP$ w każdym z zaprezentowanych przypadków. Ciągnie on niestety za sobą stworzenie słownika $DBF$, żeby móc z niego w ogóle korzystać. Dla pierwszego tekstu ($data1$) nawet przyjmując czas $0.0s$ wyszukiwania każdego wzorca musielibyśmy wykonać około $400$ wyszukań, żeby nadrobić czas przeznaczony na preprocessing $DBF$. Nasza metoda jest zatem użyteczna tylko wtedy, kiedy istnieje potrzeba wielokrotnego szukania wzorców w tekście. Oczywiście $KMP$ znowu wygrywa pod względem pamięci.  
- algorytm tworzenia $DBF$ dla dużych tekstów potrafi osiągnąć bardzo duże rozmiary pamięciowe ($O(nlogn)$). Dla zbyt dużych tekstów może więc okazać się bezużyteczny, ponieważ zająłby zbyt dużo pamięci.  
  
Z powyższych wniosków płynie jeden wniosek ogólny - algorytm wyszukiwania wzorców używający $DBF$ jest użyteczny tylko wtedy, kiedy nie przejmujemy się nadto pamięcią i zamierzamy wykonywać bardzo dużo wyszukiwań. W każdym innym wypadku lepiej jest używać bazowych metod na wyszukiwanie wzorców takich jak $KMP$ czy metoda korzystająca z automatów.